<a href="https://colab.research.google.com/github/pinilDissanayaka/Google-Palm2-RAG-application/blob/main/Simple%20RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip uninstall sentence-transformers
!pip install sentence-transformers==2.2.2
!pip install faiss-gpu
!pip install langchain
!pip install langchain_community
!pip install langchain_groq
!pip install langchain_huggingface

Found existing installation: sentence-transformers 2.2.2
Uninstalling sentence-transformers-2.2.2:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/sentence_transformers-2.2.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/sentence_transformers/*
Proceed (Y/n)? n
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2


In [1]:
import os
from langchain_groq import ChatGroq
from langchain.document_loaders.web_base import WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [2]:
from google.colab import userdata
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')

In [12]:
llm=ChatGroq(model="llama3-70b-8192",
             temperature=0.6)

In [4]:
loader=WebBaseLoader("https://www.aljazeera.com/news/2024/8/12/paris-2024-olympics-close-los-angeles-takes-torch-five-key-takeaways")
data=loader.load()

print(data)

[Document(metadata={'source': 'https://www.aljazeera.com/news/2024/8/12/paris-2024-olympics-close-los-angeles-takes-torch-five-key-takeaways', 'title': 'Paris Olympics 2024 close, Los Angeles takes torch: Five key takeaways | Paris Olympics 2024 News | Al Jazeera', 'description': 'As the Paris Olympics ends with a spectacular closing ceremony, we take a look at some of the key moments.', 'language': 'en'}, page_content='Paris Olympics 2024 close, Los Angeles takes torch: Five key takeaways | Paris Olympics 2024 News | Al Jazeera\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip linksSkip to Contentplay Live Show navigation menuNavigation menuNewsShow more news sectionsMiddle EastAfricaAsiaUS & CanadaLatin AmericaEuropeAsia PacificWar on GazaUS ElectionOpinionSportVideoMoreShow more sectionsFeaturesUkraine warEconomyClimate CrisisInvestigationsInteractivesIn PicturesScience & TechnologyPodcastsplay Live Click here to searchsearchNews|Paris Olympics 2024Paris Olympics 2024 close, Los Ang

In [5]:
embeddingModel = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
vectorDAtabase=FAISS.from_documents(data, embeddingModel)

In [7]:
retriever=vectorDAtabase.as_retriever()

In [8]:
template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


chatPromptTemplate = ChatPromptTemplate.from_template(template)

In [9]:
chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | chatPromptTemplate
    | llm
    | StrOutputParser()
)

In [14]:
chain.invoke("Who gave the Olympic flag to US gymnast Simone Biles?")

'According to the context, LA Mayor Karen Bass gave the Olympic flag to US gymnast Simone Biles to hold as H E R sang the US national anthem live at the Stade de France.'

In [15]:
llm.invoke("gave the Olympic flag to US gymnast Simone Biles?")

AIMessage(content='The Olympic flag was passed to US gymnast Simone Biles by... **IOC President Thomas Bach**!', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 23, 'total_tokens': 45, 'completion_time': 0.065931051, 'prompt_time': 0.004104562, 'queue_time': None, 'total_time': 0.07003561300000001}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'stop', 'logprobs': None}, id='run-4be6a982-0864-409b-a24e-53a8adfe1b79-0', usage_metadata={'input_tokens': 23, 'output_tokens': 22, 'total_tokens': 45})